In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from transformers import T5Tokenizer, TFAutoModelForSeq2SeqLM
import pandas as pd
import ast

In [ ]:
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/Project 2/cleaned_data.csv')
data.head()

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209110 entries, 0 to 209109
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   headline             209110 non-null  object
 1   title                209110 non-null  object
 2   text                 209110 non-null  object
 3   lemmatized headline  209110 non-null  object
 4   lemmatized text      209110 non-null  object
 5   input_ids            209110 non-null  object
 6   attention_mask       209110 non-null  object
 7   labels               209110 non-null  object
dtypes: object(8)
memory usage: 12.8+ MB


In [ ]:
df_sampled = data.sample(n=50000, random_state=42)  # Selects 50,000 random rows
df_sampled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 151785 to 68967
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   headline             50000 non-null  object
 1   title                50000 non-null  object
 2   text                 50000 non-null  object
 3   lemmatized headline  50000 non-null  object
 4   lemmatized text      50000 non-null  object
 5   input_ids            50000 non-null  object
 6   attention_mask       50000 non-null  object
 7   labels               50000 non-null  object
dtypes: object(8)
memory usage: 3.4+ MB


In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))  # Should show GPU details


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Convert string representations of lists back to lists for sampled data
df_sampled["input_ids"] = df_sampled["input_ids"].apply(ast.literal_eval)
df_sampled["attention_mask"] = df_sampled["attention_mask"].apply(ast.literal_eval)
df_sampled["labels"] = df_sampled["labels"].apply(ast.literal_eval)

def encode_data(sample):
    return {
        "input_ids": tf.convert_to_tensor(sample["input_ids"], dtype=tf.int32),
        "attention_mask": tf.convert_to_tensor(sample["attention_mask"], dtype=tf.int32),
        "labels": tf.convert_to_tensor(sample["labels"], dtype=tf.int32),
    }

# Create dataset from sampled data
dataset = tf.data.Dataset.from_generator(
    lambda: (encode_data(row) for _, row in df_sampled.iterrows()),
    output_signature={
        "input_ids": tf.TensorSpec(shape=(None,), dtype=tf.int32),
        "attention_mask": tf.TensorSpec(shape=(None,), dtype=tf.int32),
        "labels": tf.TensorSpec(shape=(None,), dtype=tf.int32),
    }
)

# Batch and shuffle
BATCH_SIZE = 8  # Adjust as needed
dataset = dataset.shuffle(5000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 1


In [ ]:
model_name = "t5-small"  # Choose "t5-base" for better results
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss_fn)

In [ ]:
model.fit(dataset, epochs=2)


Epoch 1/2
6250/6250 [==============================] - 2618s 412ms/step - loss: 1.6532
Epoch 2/2
3903/6250 [=================>............] - ETA: 16:04 - loss: 1.3913

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Project 2/t5_tf_text_summarization")
tokenizer.save_pretrained("/content/drive/MyDrive/Project 2/t5_tf_text_summarization")

('./t5_tf_text_summarization/tokenizer_config.json',
 './t5_tf_text_summarization/special_tokens_map.json',
 './t5_tf_text_summarization/spiece.model',
 './t5_tf_text_summarization/added_tokens.json')

In [ ]:
def summarize(text):
    inputs = tokenizer(text, return_tensors="tf", max_length=512, truncation=True)
    summary_ids = model.generate(**inputs, max_length=150, num_beams=5)
    return tokenizer.decode(summary_ids[0].numpy(), skip_special_tokens=True)

test_text = "Artificial Intelligence (AI) has emerged as one of the most transformative technologies of the 21st century. It is reshaping industries by automating processes, enhancing decision-making, and improving efficiency. In the healthcare sector, AI-driven tools assist doctors in diagnosing diseases, predicting patient outcomes, and even performing robotic surgeries. Financial institutions leverage AI for fraud detection, algorithmic trading, and customer service chatbots. Meanwhile, in the entertainment industry, AI personalizes content recommendations, enhances gaming experiences, and even assists in scriptwriting.Despite these advancements, AI also presents challenges. Concerns over data privacy and security have been raised as AI systems collect and analyze massive amounts of user data. Additionally, job displacement due to automation is a growing concern, with many fearing that AI will replace human workers in various fields. Policymakers and researchers emphasize the importance of developing ethical AI frameworks to ensure responsible deployment and mitigate potential risks.As AI continues to evolve, it is crucial to balance its benefits with ethical considerations. With the right policies and safeguards in place, AI has the potential to drive innovation while addressing societal concerns."
print("Summary:", summarize(test_text))

Summary: . AI is reshaping industries by automating processes, enhancing decision-making, and improving efficiency. AI is reshaping industries by automating processes, enhancing decision-making, and improving efficiency. In the healthcare sector, AI-driven tools assist doctors in diagnosing diseases, predicting patient outcomes, and even performing robotic surgeries. In the entertainment industry, AI personalizes content recommendations, enhances gaming experiences, and even assists in scriptwriting.


In [ ]:
def summarize(text):
    inputs = tokenizer(text, return_tensors="tf", max_length=512, truncation=True)
    summary_ids = model.generate(**inputs, max_length=150, num_beams=5)
    return tokenizer.decode(summary_ids[0].numpy(), skip_special_tokens=True)
sample_text = """The global economy has been facing significant challenges over the past few years, driven by a combination of geopolitical tensions, supply chain disruptions, and rising inflationary pressures.
Governments and central banks worldwide have been struggling to balance economic growth with inflation control, often resorting to measures such as adjusting interest rates, modifying fiscal policies, and implementing stimulus packages.

One of the major concerns is the ongoing trade war between some of the world's largest economies, which has led to tariffs, export restrictions, and strained international relations. Additionally, the lingering effects of the COVID-19 pandemic continue to disrupt global supply chains, leading to shortages in key industries such as semiconductor manufacturing, consumer electronics, and automobile production.

Stock markets across the globe have witnessed increased volatility, with tech stocks experiencing major fluctuations due to concerns over regulatory changes and declining revenues. Investors are closely watching the Federal Reserve’s policy decisions, as any indication of aggressive interest rate hikes could further impact global equity markets. Meanwhile, inflation has reached multi-decade highs in several countries, pushing consumers to reduce discretionary spending.

To counteract these economic headwinds, policymakers are considering various measures, including tax cuts, increased government spending on infrastructure, and targeted subsidies to support struggling businesses. However, the effectiveness of these measures depends on multiple factors, including consumer confidence, labor market stability, and international cooperation.

Despite these challenges, some sectors have shown resilience, including renewable energy, artificial intelligence, and healthcare technology. Experts believe that while short-term uncertainties persist, long-term growth prospects remain optimistic, provided that economies can navigate the current crisis effectively through coordinated policy efforts and sustainable economic reforms."""
print("Summary:", summarize(sample_text))

Summary: are facing significant challenges over the past few years, driven by geopolitical tensions, supply chain disruptions, and rising inflationary pressures. Despite these challenges, some sectors have shown resilience, including renewable energy, artificial intelligence, and healthcare technology.
